# Groupping and aggregation



In [2]:
import pandas as pd

In [3]:
url = 'https://raw.githubusercontent.com/piotrgradzinski/dap_20230114/main/day_6_pgg/emps.csv'
emps = pd.read_csv(url, sep=';', encoding='utf-8', index_col='employee_id', parse_dates=['hire_date'])
emps

,first_name,last_name,job_title,salary,hire_date,department_name,address,postal_code,city,country
employee_id,,,,,,,,,,
100,Steven,King,President,24000,1997-06-17,Executive,2004 Charade Rd,98199,Seattle,United States of America
101,Neena,Kochhar,Administration Vice President,17000,1999-09-21,Executive,2004 Charade Rd,98199,Seattle,United States of America
102,Lex,De Haan,Administration Vice President,17000,2003-01-13,Executive,2004 Charade Rd,98199,Seattle,United States of America
103,Alexander,Hunold,Programmer,9000,2000-01-03,IT,2014 Jabberwocky Rd,26192,Southlake,United States of America
104,Bruce,Ernst,Programmer,6000,2001-05-21,IT,2014 Jabberwocky Rd,26192,Southlake,United States of America
...,...,...,...,...,...,...,...,...,...,...
202,Pat,Fay,Marketing Representative,6000,2007-08-17,Marketing,147 Spadina Ave,M5V 2L7,Toronto,Canada
203,Susan,Mavris,Human Resources Representative,6500,2004-06-07,Human Resources,8204 Arthur St,NaN,London,United Kingdom
204,Hermann,Baer,Public Relations Representative,10000,2004-06-07,Public Relations,Schwanthalerstr. 7031,80925,Munich,Germany


## Aggregate functions

- sum
- mean
- median
- min
- max
- std

Funtions we can use that are python based or from NumPy library, etc.

### Difference between mean and median

The mean (average) of a data set is found by adding all numbers in the data set and then dividing by the number of values in the set. 

The median is the middle value when a data set is ordered from least to greatest. 

The mode is the number that occurs most often in a data set.

In [4]:
emps.salary.sum()

691400

In [6]:
emps.salary.mean()

6461.682242990654

In [8]:
emps.salary.median()

6200.0

We can execute aggregate functions on a DataFrame (instead of Series) and aggregate function will try to calculate the value where it's possible.

In [10]:
emps.sum()

/var/folders/__/0gvd5dsj22b9cbv4hzvln3pw0000gn/T/ipykernel_9205/4011394593.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  emps.sum()


first_name    StevenNeenaLexAlexanderBruceDavidValliDianaNan...
last_name     KingKochharDe HaanHunoldErnstAustinPataballaLo...
job_title     PresidentAdministration Vice PresidentAdminist...
salary                                                   691400
dtype: object

So can say that we want to apply an aggregate function to certain columns and get the results as a DataFrame (or Series). We can apply several functions for one column, we can aggregate on multiple columns.

To do that we will use [`agg()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html) (`aggregate`).

In [12]:
emps.agg({
    'salary': 'mean',
    'hire_date': 'median'
})

salary               6461.682243
hire_date    2007-09-28 00:00:00
dtype: object

We can provide aggregate functions as strings (or a list of strings).

In [13]:
emps.agg({
    'salary': ['min', 'max', 'mean', 'median', 'sum', 'std'],
    'hire_date': ['min', 'max', 'median']
})

,salary,hire_date
min,2100.000000,1987-09-17
max,24000.000000,2011-02-06
mean,6461.682243,NaT
median,6200.000000,2007-09-28
sum,691400.000000,NaT
std,3909.365746,NaT


We can provide python or NumPy functions.

In [15]:
import numpy as np

In [17]:
emps.agg({
    'salary': [min, max, np.median, 'mean', 'std'],
    'hire_date': [min, max, 'median'],
})

,salary,hire_date
min,2100.000000,1987-09-17
max,24000.000000,2011-02-06
median,6200.000000,2007-09-28
mean,6461.682243,NaT
std,3909.365746,NaT


## Grouping

I can do that manually by filtering the data and then using aggregate functions- but it's not a best solution, especially if we want to have aggregate results for all possible values. 

For that we will use grouping - we will group all rows by certain column, for example `city`. I will have groups of rows and for each group city value will be the same, for example 'all employess from London, Seattle, etc.'.

We will use [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) for that. 

In [22]:
emps[emps.city == 'Seattle'].salary.agg([min, max])

min     2500
max    24000
Name: salary, dtype: int64

When use `groupby` function we will get a new `DataFrameGroupBy` object that will contain all the rows organized into groups.

In [26]:
groups = emps.groupby('city')
groups

In [27]:
len(groups)  # without NaN

7

In [29]:
emps.city.unique()

array(['Seattle', 'Southlake', 'South San Francisco', 'Oxford', nan,
       'Toronto', 'London', 'Munich'], dtype=object)

In [31]:
groups.size()

city
London                  1
Munich                  1
Oxford                 34
Seattle                18
South San Francisco    45
Southlake               5
Toronto                 2
dtype: int64

In [32]:
groups.get_group('Toronto')

,first_name,last_name,job_title,salary,hire_date,department_name,address,postal_code,city,country
employee_id,,,,,,,,,,
201,Michael,Hartstein,Marketing Manager,13000,2006-02-17,Marketing,147 Spadina Ave,M5V 2L7,Toronto,Canada
202,Pat,Fay,Marketing Representative,6000,2007-08-17,Marketing,147 Spadina Ave,M5V 2L7,Toronto,Canada


In [34]:
for city, group in groups:
    print(f'Employees from {city}')
    for idx, emp in group.iterrows():
        print(f"\t {emp.first_name} {emp.last_name}")

Employees from London
	 Susan Mavris
Employees from Munich
	 Hermann Baer
Employees from Oxford
	 John Russell
	 Karen Partners
	 Alberto Errazuriz
	 Gerald Cambrault
	 Eleni Zlotkey
	 Peter Tucker
	 David Bernstein
	 Peter Hall
	 Christopher Olsen
	 Nanette Cambrault
	 Oliver Tuvault
	 Janette King
	 Patrick Sully
	 Allan McEwen
	 Lindsey Smith
	 Louise Doran
	 Sarath Sewall
	 Clara Vishney
	 Danielle Greene
	 Mattea Marvins
	 David Lee
	 Sundar Ande
	 Amit Banda
	 Lisa Ozer
	 Harrison Bloom
	 Tayler Fox
	 William Smith
	 Elizabeth Bates
	 Sundita Kumar
	 Ellen Abel
	 Alyssa Hutton
	 Jonathon Taylor
	 Jack Livingston
	 Charles Johnson
Employees from Seattle
	 Steven King
	 Neena Kochhar
	 Lex De Haan
	 Nancy Greenberg
	 Daniel Faviet
	 John Chen
	 Ismael Sciarra
	 Jose Manuel Urman
	 Luis Popp
	 Den Raphaely
	 Alexander Khoo
	 Shelli Baida
	 Sigal Tobias
	 Guy Himuro
	 Karen Colmenares
	 Jennifer Whalen
	 Shelley Higgins
	 William Gietz
Employees from South San Francisco
	 Matthew Wei

Exporting DataFrame to a file is simple and we can use `.to_*` methods. For example [`.to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html).

In [35]:
for city, group in groups:
    group.to_csv(f"group_{city.lower()}.csv")

## Groupping and aggregating

The most common approach is to first group data by some column and then execute aggregate function(s) on those groups.

In [36]:
groups.mean()

,salary
city,
London,6500.000000
Munich,10000.000000
Oxford,8955.882353
Seattle,8844.444444
South San Francisco,3475.555556
Southlake,5760.000000
Toronto,9500.000000


In [38]:
groups.agg({
    'salary': ['count', min, max, 'mean', 'median', 'sum', 'std'],
    'hire_date': [min, max, 'median']
})

salary                                               \
                     count    min    max          mean   median     sum   
city                                                                      
London                   1   6500   6500   6500.000000   6500.0    6500   
Munich                   1  10000  10000  10000.000000  10000.0   10000   
Oxford                  34   6100  14000   8955.882353   8900.0  304500   
Seattle                 18   2500  24000   8844.444444   8000.0  159200   
South San Francisco     45   2100   8200   3475.555556   3100.0  156400   
Southlake                5   4200   9000   5760.000000   4800.0   28800   
Toronto                  2   6000  13000   9500.000000   9500.0   19000   

                                  hire_date                                 
                             std        min        max              median  
city                                                                        
London                       NaN 2004-06-07 2004-06-07 2004-06-07 00:00:00  
Munich                       NaN 2004-06-07 2004-06-07 2004-06-07 00:00:00  
Oxford               2033.684695 2000-01-29 2011-01-04 2007-09-30 12:00:00  
Seattle              5931.295089 1987-09-17 2009-12-07 2004-10-12 00:00:00  
South San Francisco  1488.005919 2000-02-03 2011-02-06 2008-02-23 00:00:00  
Southlake            1925.616784 2000-01-03 2009-02-07 2007-06-25 00:00:00  
Toronto              4949.747468 2006-02-17 2007-08-17 2006-11-17 00:00:00

In most cases we are doing `groupby` and `agg` in the same expression. 

In [40]:
emps.groupby('job_title').agg({
    'salary': ['count', min, max, 'mean', 'median', 'sum', 'std'],
    'hire_date': [min, max, 'median']
})

salary                                  \
                                 count    min    max     mean   median   
job_title                                                                
Accountant                           5   6900   9000   7920.0   7800.0   
Accounting Manager                   1  12000  12000  12000.0  12000.0   
Administration Assistant             1   4400   4400   4400.0   4400.0   
Administration Vice President        2  17000  17000  17000.0  17000.0   
Finance Manager                      1  12000  12000  12000.0  12000.0   
Human Resources Representative       1   6500   6500   6500.0   6500.0   
Marketing Manager                    1  13000  13000  13000.0  13000.0   
Marketing Representative             1   6000   6000   6000.0   6000.0   
President                            1  24000  24000  24000.0  24000.0   
Programmer                           5   4200   9000   5760.0   4800.0   
Public Accountant                    1   8300   8300   8300.0   8300.0   
Public Relations Representative      1  10000  10000  10000.0  10000.0   
Purchasing Clerk                     5   2500   3100   2780.0   2800.0   
Purchasing Manager                   1  11000  11000  11000.0  11000.0   
Sales Manager                        5  10500  14000  12200.0  12000.0   
Sales Representative                30   6100  11500   8350.0   8200.0   
Shipping Clerk                      20   2500   4200   3215.0   3100.0   
Stock Clerk                         20   2100   3600   2785.0   2700.0   
Stock Manager                        5   5800   8200   7280.0   7900.0   

                                                      hire_date             \
                                    sum          std        min        max   
job_title                                                                    
Accountant                        39600   766.159252 1998-03-07 2009-12-07   
Accounting Manager                12000          NaN 2004-06-07 2004-06-07   
Administration Assistant           4400          NaN 1987-09-17 1987-09-17   
Administration Vice President     34000     0.000000 1999-09-21 2003-01-13   
Finance Manager                   12000          NaN 2004-08-17 2004-08-17   
Human Resources Representative     6500          NaN 2004-06-07 2004-06-07   
Marketing Manager                 13000          NaN 2006-02-17 2006-02-17   
Marketing Representative           6000          NaN 2007-08-17 2007-08-17   
President                         24000          NaN 1997-06-17 1997-06-17   
Programmer                        28800  1925.616784 2000-01-03 2009-02-07   
Public Accountant                  8300          NaN 2004-06-07 2004-06-07   
Public Relations Representative   10000          NaN 2004-06-07 2004-06-07   
Purchasing Clerk                  13900   238.746728 2005-05-18 2009-08-10   
Purchasing Manager                11000          NaN 2004-12-07 2004-12-07   
Sales Manager                     61000  1524.795068 2000-01-29 2009-10-15   
Sales Representative             250500  1524.229596 2000-02-23 2011-01-04   
Shipping Clerk                    64300   547.986746 2000-02-03 2010-01-13   
Stock Clerk                       55700   453.379126 2005-07-14 2011-02-06   
Stock Manager                     36400  1066.302021 2005-05-01 2009-11-16   

                                                     
                                             median  
job_title                                            
Accountant                      2007-09-28 00:00:00  
Accounting Manager              2004-06-07 00:00:00  
Administration Assistant        1987-09-17 00:00:00  
Administration Vice President   2001-05-18 00:00:00  
Finance Manager                 2004-08-17 00:00:00  
Human Resources Representative  2004-06-07 00:00:00  
Marketing Manager               2006-02-17 00:00:00  
Marketing Representative        2007-08-17 00:00:00  
President                       1997-06-17 00:00:00  
Programmer                      2007-0

In [47]:
emps.groupby(emps.hire_date.dt.year).agg({
    'salary': ['count', min, max, 'mean', 'median', 'sum', 'std'],
    'hire_date': [min, max, 'median']
})

salary                                                            \
           count    min    max          mean   median     sum          std   
hire_date                                                                    
1987           1   4400   4400   4400.000000   4400.0    4400          NaN   
1997           1  24000  24000  24000.000000  24000.0   24000          NaN   
1998           1   7800   7800   7800.000000   7800.0    7800          NaN   
1999           1  17000  17000  17000.000000  17000.0   17000          NaN   
2000           6   2800  10500   6950.000000   6600.0   41700  2642.536660   
2001           1   6000   6000   6000.000000   6000.0    6000          NaN   
2003           1  17000  17000  17000.000000  17000.0   17000          NaN   
2004           7   6500  12000   9828.571429  10000.0   68800  2038.556913   
2005           4   3100   7900   4525.000000   3550.0   18100  2260.346581   
2006          10   3300  14000   8600.000000   9250.0   86000  3744.329045   
2007          28   2500  13500   6460.714286   6250.0  180900  3304.019918   
2008          22   2500  10000   4740.909091   3150.0  104300  2691.903043   
2009          18   2100  11000   4938.888889   3600.0   88900  2822.209999   
2010           4   2200   7200   4525.000000   4350.0   18100  2499.833328   
2011           2   2200   6200   4200.000000   4200.0    8400  2828.427125   

           hire_date                                 
                 min        max              median  
hire_date                                            
1987      1987-09-17 1987-09-17 1987-09-17 00:00:00  
1997      1997-06-17 1997-06-17 1997-06-17 00:00:00  
1998      1998-03-07 1998-03-07 1998-03-07 00:00:00  
1999      1999-09-21 1999-09-21 1999-09-21 00:00:00  
2000      2000-01-03 2000-04-21 2000-02-13 00:00:00  
2001      2001-05-21 2001-05-21 2001-05-21 00:00:00  
2003      2003-01-13 2003-01-13 2003-01-13 00:00:00  
2004      2004-06-07 2004-12-07 2004-06-07 00:00:00  
2005      2005-05-01 2005-10-17 2005-06-15 12:00:00  
2006      2006-01-27 2006-10-01 2006-04-07 00:00:00  
2007      2007-01-05 2007-12-24 2007-07-05 12:00:00  
2008      2008-01-24 2008-12-09 2008-04-23 12:00:00  
2009      2009-01-14 2009-12-19 2009-06-07 00:00:00  
2010      2010-01-13 2010-04-21 2010-02-14 12:00:00  
2011      2011-01-04 2011-02-06 2011-01-20 12:00:00

In [46]:
emps.groupby(emps.hire_date.dt.strftime('%Y-%m')).agg({
    'salary': ['count', min, max, 'mean', 'median', 'sum', 'std'],
    'hire_date': [min, max, 'median']
})

salary                                                      \
           count    min    max     mean   median    sum          std   
hire_date                                                              
1987-09        1   4400   4400   4400.0   4400.0   4400          NaN   
1997-06        1  24000  24000  24000.0  24000.0  24000          NaN   
1998-03        1   7800   7800   7800.0   7800.0   7800          NaN   
1999-09        1  17000  17000  17000.0  17000.0  17000          NaN   
2000-01        2   9000  10500   9750.0   9750.0  19500  1060.660172   
...          ...    ...    ...      ...      ...    ...          ...   
2010-01        2   2600   7200   4900.0   4900.0   9800  3252.691193   
2010-03        1   2200   2200   2200.0   2200.0   2200          NaN   
2010-04        1   6100   6100   6100.0   6100.0   6100          NaN   
2011-01        1   6200   6200   6200.0   6200.0   6200          NaN   
2011-02        1   2200   2200   2200.0   2200.0   2200          NaN   

           hire_date                                 
                 min        max              median  
hire_date                                            
1987-09   1987-09-17 1987-09-17 1987-09-17 00:00:00  
1997-06   1997-06-17 1997-06-17 1997-06-17 00:00:00  
1998-03   1998-03-07 1998-03-07 1998-03-07 00:00:00  
1999-09   1999-09-21 1999-09-21 1999-09-21 00:00:00  
2000-01   2000-01-03 2000-01-29 2000-01-16 00:00:00  
...              ...        ...                 ...  
2010-01   2010-01-13 2010-01-24 2010-01-18 12:00:00  
2010-03   2010-03-08 2010-03-08 2010-03-08 00:00:00  
2010-04   2010-04-21 2010-04-21 2010-04-21 00:00:00  
2011-01   2011-01-04 2011-01-04 2011-01-04 00:00:00  
2011-02   2011-02-06 2011-02-06 2011-02-06 00:00:00  

[61 rows x 10 columns]

We can group by multiple columns at one time.

In [50]:
data = emps.groupby(['city', 'job_title']).agg({
    'salary': ['count', min, max, 'mean', 'median', 'sum', 'std'],
    'hire_date': [min, max, 'median']
})
data

salary                \
                                                     count    min    max   
city                job_title                                              
London              Human Resources Representative       1   6500   6500   
Munich              Public Relations Representative      1  10000  10000   
Oxford              Sales Manager                        5  10500  14000   
                    Sales Representative                29   6100  11500   
Seattle             Accountant                           5   6900   9000   
                    Accounting Manager                   1  12000  12000   
                    Administration Assistant             1   4400   4400   
                    Administration Vice President        2  17000  17000   
                    Finance Manager                      1  12000  12000   
                    President                            1  24000  24000   
                    Public Accountant                    1   8300   8300   
                    Purchasing Clerk                     5   2500   3100   
                    Purchasing Manager                   1  11000  11000   
South San Francisco Shipping Clerk                      20   2500   4200   
                    Stock Clerk                         20   2100   3600   
                    Stock Manager                        5   5800   8200   
Southlake           Programmer                           5   4200   9000   
Toronto             Marketing Manager                    1  13000  13000   
                    Marketing Representative             1   6000   6000   

                                                                            \
                                                             mean   median   
city                job_title                                                
London              Human Resources Representative    6500.000000   6500.0   
Munich              Public Relations Representative  10000.000000  10000.0   
Oxford              Sales Manager                    12200.000000  12000.0   
                    Sales Representative              8396.551724   8400.0   
Seattle             Accountant                        7920.000000   7800.0   
                    Accounting Manager               12000.000000  12000.0   
                    Administration Assistant          4400.000000   4400.0   
                    Administration Vice President    17000.000000  17000.0   
                    Finance Manager                  12000.000000  12000.0   
                    President                        24000.000000  24000.0   
                    Public Accountant                 8300.000000   8300.0   
                    Purchasing Clerk                  2780.000000   2800.0   
                    Purchasing Manager               11000.000000  11000.0   
South San Francisco Shipping Clerk                    3215.000000   3100.0   
                    Stock Clerk                       2785.000000   2700.0   
                    Stock Manager                     7280.000000   7900.0   
Southlake           Programmer                        5760.000000   4800.0   
Toronto             Marketing Manager                13000.000000  13000.0   
                    Marketing Representative          6000.000000   6000.0   

                                                                          \
                                                        sum          std   
city                job_title                                              
London              Human Resources Representative     6500          NaN   
Munich              Public Relations Representative   10000          NaN   
Oxford              Sales Manager                     61000  1524.795068   
                    Sales Representative             243500  1529.351580   
Seattle             Accountant                        39600   766.159252   
                    Accounting Manager                

If we want to read a particular column with a multi-level index we need to provide a tuple containg names of levels in the hierarchy from which column I want to take the data.

In [52]:
data[('salary', 'median')]

city                 job_title                      
London               Human Resources Representative      6500.0
Munich               Public Relations Representative    10000.0
Oxford               Sales Manager                      12000.0
                     Sales Representative                8400.0
Seattle              Accountant                          7800.0
                     Accounting Manager                 12000.0
                     Administration Assistant            4400.0
                     Administration Vice President      17000.0
                     Finance Manager                    12000.0
                     President                          24000.0
                     Public Accountant                   8300.0
                     Purchasing Clerk                    2800.0
                     Purchasing Manager                 11000.0
South San Francisco  Shipping Clerk                      3100.0
                     Stock Clerk                   

We can use the same approach for rows which have multi-level index.

In [53]:
data.loc[('Oxford', 'Sales Representative')]

salary     count                      29
           min                      6100
           max                     11500
           mean              8396.551724
           median                 8400.0
           sum                    243500
           std                1529.35158
hire_date  min       2000-02-23 00:00:00
           max       2011-01-04 00:00:00
           median    2007-12-15 00:00:00
Name: (Oxford, Sales Representative), dtype: object

If we want to get a value of a cell from such data frame where both rows and column have multi-level indexes

In [54]:
data.loc[('Oxford', 'Sales Representative'), ('salary', 'count')]

29

In [56]:
data.iloc[4, 2]

9000